In [1]:
from bertviz import head_view
#from transformers import BertTokenizer, BertModel

In [2]:
import os

In [3]:
# Adding module to sys path
import sys
sys.path.append("../MEDeA/")
# RNN imports
import medea
from numpy import mean
from medea import torch, numpy as np

Changed the global seed : 0 -> 0


In [4]:
from medea.models.composite_model import CompositeModel
from medea.training.model_trainer import ModelTrainer
from medea.inputs.data.build_and_embed.data_builder import DatasetBuilderEmbedder
from medea.inputs.data.read import MedeaDatasetReader, get_all_phones, get_all_phones_to_frequency
from medea.inputs.parameters.embeddings import EmbeddingParams, EmbeddingParamsOneLang
from medea.utils.shuffling import ShuffleType
from medea.utils import BatchInfo

In [5]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

## Data

In [6]:
path_params = "../MEDeA/datasets/file_parameters/default/"
path_data = "../MEDeA/datasets/romance_languages/cog/"
langs = ["es", "it", "la"]
test_data = MedeaDatasetReader(folder_path=path_data, langs=langs, name="Triple", phonetized=True)

#lang_pairs = [["es", "it", "la"]]

all_langs = ["es", "it", "la"]
lang_pairs = [all_langs,
              ["it", "la"], ["es", "it"], ["es", "la"],
             ]
tokens = ["EOW", "es", "it", "la"]
phones = [p for p in set(get_all_phones(path_data, lang_pairs)) if p not in tokens]

all_words = sorted(phones)
vocab_dim = len(all_words) + len(tokens)

embed_1_params = EmbeddingParamsOneLang.from_json_file(f"{path_params}embed_1_parameters.json")
embed_params = {}
for lang in ["es", "it", "la"]:
    embed_params[lang] = embed_1_params
    embed_params[lang].phone_embedding.phone_categories = list(all_words)
    embed_params[lang].pseudotoken_embedding.list = tokens
embed_params = EmbeddingParams(embed_params)

# Used for finetuning - we keep all the pairs
train_set = DatasetBuilderEmbedder.build(data_list=[test_data], parameters=embed_params)

[INFO] By setting the padding length to -1, you mean that you do not want padding.
[INFO] 3 duplicate words present in set 'Triple' were removed.


In [7]:
batches = train_set.to_batch(ShuffleType.DESCENDING, batch_size=1, 
                             langs=langs, lang_of_reference_for_shuffling=langs[0])
batch_info = train_set.get_batch_info()

In [64]:
batch = batches[100]

In [65]:
in_lang = langs[0]
out_lang = langs[1]

In [66]:
batch

defaultdict(list,
            {'it': [tensor([68., 15., 27., 11., 28., 30., 13.,  1., 21., 23., 66.])],
             'la': [tensor([69., 15., 10., 27., 50., 28., 30., 50.,  1., 21., 57., 28., 66.])],
             'es': [tensor([67., 30., 55., 54., 11., 28., 30., 13.,  0., 21., 23., 66.])]})

## Model

In [67]:
seed = 0
epoch = 20
num_head = 4
num_layers = 2

model_path = os.path.expanduser("/Users/cfourrie/Desktop/Experiments_synthesis_Y1/Exp5/" + \
                                f"Transformer{num_head}head{num_layers}layer_20_36_{seed}" + \
                                f"/results/experiment/models/epoch_{epoch}")
model = CompositeModel.load(model_path)


In [68]:
predictions, attn = model.predict(batch, batch_info)

In [69]:
local_prediction = [
    train_set[out_lang].ix_to_item[int(c)]
    for c in predictions["decoder"][f"{in_lang}_{out_lang}"][0][0]
]
local_target = [
    train_set[out_lang].ix_to_item[int(c)]
    for c in batch[out_lang][0]
]
local_input = [
    train_set[in_lang].ix_to_item[int(c)]
    for c in batch[in_lang][0]
]

local_self_attn = []
local_enc_attn = []
local_attn = []
for l_num in range(num_layers):
    local_self_attn.append(torch.stack(
        [attn[f"{in_lang}_{out_lang}"][0][f"decoder_self_attn/layer{l_num}_head{h}"]
         for h in range(num_head)]
    ).unsqueeze(0))

    local_enc_attn.append(torch.stack(
        [attn[f"{in_lang}_{out_lang}"][0][f"encoder_attn/layer{l_num}_head{h}"]
         for h in range(num_head)], dim=0).transpose(0, 1))
                          
    local_attn.append(torch.stack(
        [attn[f"{in_lang}_{out_lang}"][0][f"decoder_attn/layer{l_num}_head{h}"]
         for h in range(num_head)]
    ).unsqueeze(0))

In [70]:
print(
    "".join([train_set[out_lang].ix_to_item[int(c)] for c in batch[in_lang][0]][1:-1]), 
    "=>", 
    "".join([train_set[out_lang].ix_to_item[int(c)] for c in batch[out_lang][0]][1:-1]))

enc = False
self = False

if enc:
    print("Encoder attention")
    head_view(local_enc_attn, local_input, local_input, prettify_tokens=False) 
else:
    if self:
        print("Decoder self attention")
        head_view(local_self_attn, local_prediction[1:], local_prediction, prettify_tokens=False)
    else:
        print("Decoder attention")
        head_view(local_attn, local_prediction[1:], local_input, prettify_tokens=False)



tʃɾistjano => kristja:no
Decoder attention


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>